In [1]:
import cv2 as cv
import numpy as np
from skimage import measure, morphology
from skimage.color import label2rgb
from skimage.measure import regionprops
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import glob
import matplotlib.pyplot  as plt
import os
import json
from typing import List, Dict


In [2]:
minLineLength = 2
maxLineGap = 4
def lines_extraction(gray: List[int]) -> List[int]:
    """
    this function extracts the lines from the binary image. Cleaning process.
    """
    edges = cv.Canny(gray, 100, 150)
    lines = cv.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength, maxLineGap)
    return lines

In [3]:
def get_file_name(file):
    base=os.path.basename(file)
    fn=os.path.splitext(base)[0]
    abs_path=os.path.abspath(file)
    return fn,abs_path

In [4]:
def pre_process(Orig_path):
    for file in glob.glob(Orig_path):
        fn,_=get_file_name(file)
        image=cv.imread(file)
        image=cv.resize(image,(1280,1280))
        originalGray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        img = cv.threshold(originalGray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
        #kernel=np.zeros((3,3),np.uint8)
        #dilate=cv.dilate(img,kernel,iterations=2)
        cv.imwrite("/home/vijaysn/Documents/connected_compnent/single_signatures/preprocess/"+fn+".png",img)



In [5]:
Orig_path="/home/vijaysn/Documents/connected_compnent/single_signatures/ip/*.*"
pre_process(Orig_path)

In [4]:
def contours(path):
    for file in glob.glob(path):
        fn,_=get_file_name(file)
        _,base=get_file_name(file)
        img1=cv.imread(file,0)                  
        blur = cv.GaussianBlur(img1, (3,3), 0,0,cv.BORDER_DEFAULT)
        retval, thresh_gray = cv.threshold(img1, thresh=100, maxval=255, type=cv.THRESH_BINARY_INV)
        canny = cv.Canny(thresh_gray, 150, 120)
        kernel = np.ones((55, 55), np.uint8)
        closing = cv.morphologyEx(canny, cv.MORPH_CLOSE, kernel)
        contours, hierarchy = cv.findContours(closing, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
        total_contours=len(contours)
        return total_contours
            
            
        
            



In [ ]:
def remove_lines(preoutput_path):
      for file in glob.glob(preoutput_path):
        fn,_=get_file_name(file)
        image=cv.imread(file,0)
        lines=lines_extraction(image)
        try:
            if(lines.any()>0):
                  thresh = cv.threshold(image, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]
                  # Detect horizontal lines
                  horizontal_kernel = cv.getStructuringElement(cv.MORPH_RECT, (30,1))
                  detected_lines = cv.morphologyEx(thresh, cv.MORPH_OPEN, horizontal_kernel, iterations=2)
                  cnts = cv.findContours(detected_lines, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
                  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
                  for c in cnts:
                        cv.drawContours(image, [c], -1, (255,255,255), 2)
                  vertical_kernel = cv.getStructuringElement(cv.MORPH_RECT, (1,30))
                  detected_lines = cv.morphologyEx(thresh, cv.MORPH_OPEN, vertical_kernel, iterations=2)
                  cnts = cv.findContours(detected_lines, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
                  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
                  for c in cnts:
                   cv.drawContours(image, [c], -1, (255,255,255), 2)
                  cv.imwrite("/home/vijaysn/Documents/connected_compnent/single_signatures/pre1/"+fn+'.png',image)
        except:
            pass

In [ ]:
preoutput_path="/home/vijaysn/Documents/connected_compnent/single_signatures/preprocess/*.*"
remove_lines(preoutput_path)

In [11]:
def remove_blobs(pre_path):
    for file in glob.glob(pre_path):
        fn,_=get_file_name(file)
        img=cv.imread(file,0)
        blobs = img > img.mean()
        blobs_labels = measure.label(blobs, background=1)
        image_label_overlay = label2rgb(blobs_labels, image=img)
        regions = regionprops(blobs_labels)
        totalArea = 0
        count = 0
        for region in regions:
            if (region.area >10):
                totalArea += region.area
                count += 1
        minSize = (((totalArea/count)/60) *100) +80
        smallObjectsRemoved = morphology.remove_small_objects(blobs_labels, min_size=minSize, connectivity=1)
        plt.imsave("/home/vijaysn/Documents/connected_compnent/single_signatures/preversion/"+fn+".png", smallObjectsRemoved)
        preversion_path="/home/vijaysn/Documents/connected_compnent/single_signatures/preversion/"+fn+".png"
        preoutput_path="/home/vijaysn/Documents/connected_compnent/single_signatures/preoutput/"+fn+".png"
        img=cv.imread(preversion_path,0)
        img = cv.threshold(img,127,255,cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
        cv.imwrite(preoutput_path, img)
        noise=contours(preoutput_path)
        while True:
            if noise >4:
                minSize=minSize+50
                smallObjectsRemoved = morphology.remove_small_objects(blobs_labels, min_size=minSize, connectivity=1)
                plt.imsave("/home/vijaysn/Documents/connected_compnent/single_signatures/preversion/"+fn+".png", smallObjectsRemoved)
                preversion_path="/home/vijaysn/Documents/connected_compnent/single_signatures/preversion/"+fn+".png"
                preoutput_path="/home/vijaysn/Documents/connected_compnent/single_signatures/preoutput/"+fn+".png"
                img=cv.imread(preversion_path,0)
                img = cv.threshold(img,127,255,cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
                cv.imwrite(preoutput_path, img)
                noise=contours(preoutput_path)
             
            else:
                break
                





In [12]:
pre_path="/home/vijaysn/Documents/connected_compnent/single_signatures/pre1/*.*"
remove_blobs(pre_path)

In [9]:



def bounded_box():
    pre2_path="/home/vijaysn/Documents/connected_compnent/single_signatures/preoutput/*.*"
    Orig_path="/home/vijaysn/Documents/connected_compnent/single_signatures/ip/*.*"
    f=open('/home/vijaysn/Documents/modified/size.txt','a')

    for file in glob.glob(pre2_path):
        fn,_=get_file_name(file)
        for file1 in  glob.glob(Orig_path):
            fn1,_=get_file_name(file1)
            if(fn==fn1):
                img = cv.imread(file,0)
                orig_image=cv.imread(file1,cv.COLOR_BGR2RGB)
                orig_image=cv.resize(orig_image,(1280,1280))
                mask = np.ones(orig_image.shape[:2], dtype="uint8") * 255
                retval, thresh_gray = cv.threshold(img, thresh=100, maxval=255, type=cv.THRESH_BINARY_INV)
                canny = cv.Canny(thresh_gray, 150, 120)
                kernel = np.ones((55, 55), np.uint8)
                closing = cv.morphologyEx(canny, cv.MORPH_CLOSE, kernel)
                contours, hierarchy = cv.findContours(closing, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
                total_contours=len(contours)
                mx = (0,0,0,0)
                min_area=0
                mx_area=150000
                for cont in contours:
                    x,y,w,h = cv.boundingRect(cont)
                    area = w*h
                    if area > min_area and area < mx_area:
                        mx = x,y,w,h
                        min_area = area
                    x,y,w,h = mx

                boxed =  cv.rectangle(orig_image,(x,y),(x+w,y+h),(200,0,0), 2)
                signature = img[y: y+h, x: x+w] 
                mask[y: y+h, x: x+w] = signature # copied title contour onto the blank image
                img[y: y+h, x: x+w] = 255
                roi=mask[y:y+h, x:x+w]
                cv.imwrite("/home/vijaysn/Documents/connected_compnent/single_signatures/cropped/"+fn+'.png',roi)

                    #cv.imwrite('/home/vijaysn/Documents/modified/cropped1/'+imageName,roi)
                cv.imwrite("/home/vijaysn/Documents/connected_compnent/single_signatures/output/"+fn+'.png',mask)
                data={'File_name':fn+'.jpg','contours':len(contours),'X':x,'Y':y,'W':w,'H':h}
                f.write(json.dumps(data)+'\n')
                plt.imsave("/home/vijaysn/Documents/connected_compnent/single_signatures/region/"+fn+".png",boxed)
    f.close()
                        #boxed =  cv.rectangle(img_read,(x,y),(x+w,y+h),(200,0,0), 2)
                        #plt.imsave("/home/vijaysn/Documents/success/region/"+fn1+".png",boxed)



In [10]:
bounded_box()